# The current Jupyter Notebook will cover the full life cylce of the First Phase of the project: ETL process and "memory building"

## Scrape and extract textual content

In this step we will extract the needed data from the "Witch Cult Translations" site.

In [5]:
# Import the needed libraries for the step

import requests
from bs4 import BeautifulSoup